In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
#from datetime import datetime, timedelta
import datetime
from pprint import *

def options_chain(symbol):

    tk = yf.Ticker(symbol)
    # Expiration dates
    exps = tk.options

    # Get options for each expiration
    options = pd.DataFrame()
    for e in exps:
        opt = tk.option_chain(e)
        opt = pd.DataFrame().append(opt.calls).append(opt.puts)
        opt['expirationDate'] = e
        options = options.append(opt, ignore_index=True)

    # Bizarre error in yfinance that gives the wrong expiration date
    # Add 1 day to get the correct expiration date
    options['expirationDate'] = pd.to_datetime(options['expirationDate']) + datetime.timedelta(days = 1)
    options['dte'] = (options['expirationDate'] - datetime.datetime.today()).dt.days / 365
    
    # Boolean column if the option is a CALL
    options['CALL'] = options['contractSymbol'].str[4:].apply(
        lambda x: "C" in x)
    
    options[['bid', 'ask', 'strike']] = options[['bid', 'ask', 'strike']].apply(pd.to_numeric)
    options['mark'] = (options['bid'] + options['ask']) / 2 # Calculate the midpoint of the bid-ask
    
    # Drop unnecessary and meaningless columns
    options = options.drop(columns = ['contractSize', 'currency', 'change', 'percentChange', 'lastTradeDate', 'lastPrice'])

    return options


pprint(options_chain('VIXY'))


          contractSymbol  strike    bid    ask  volume  openInterest  \
0    VIXY201023C00017000    17.0   1.25   1.80     1.0          84.0   
1    VIXY201023C00018000    18.0   0.50   0.95   113.0         132.0   
2    VIXY201023C00018500    18.5   0.50   0.75   118.0          97.0   
3    VIXY201023C00019000    19.0   0.40   0.65    22.0         311.0   
4    VIXY201023C00019500    19.5   0.30   0.50    56.0          12.0   
..                   ...     ...    ...    ...     ...           ...   
492  VIXY220121P00055000    55.0  34.60  39.00     3.0           5.0   
493  VIXY220121P00080000    80.0  57.90  62.00     NaN          10.0   
494  VIXY220121P00084000    84.0  65.00  70.00     NaN           0.0   
495  VIXY230120C00015000    15.0   9.50  13.00     2.0           2.0   
496  VIXY230120C00032000    32.0   6.40  10.00    15.0          15.0   

     impliedVolatility  inTheMoney expirationDate       dte   CALL    mark  
0             0.777346         1.0     2020-10-24  0.01643

In [2]:
vixy = options_chain('VIXY')

In [5]:
vixy.query("expirationDate" == "2020-11-03"')

           

SyntaxError: EOL while scanning string literal (<ipython-input-5-57d338718f24>, line 1)

In [44]:
start = '2020-10-01'
end = '2020-11-15'
stock = yf.Ticker('VIXY')
s = stock.info['previousClose']
delta = 3.00
m = ( vixy['expirationDate'] > start) & (vixy['expirationDate'] <= end ) & (vixy['strike'] > (s )) & (vixy['strike'] <= (s + delta)) & (vixy['CALL'] == True)


In [45]:
c = vixy.loc[m]

In [46]:
c

,contractSymbol,strike,bid,ask,volume,openInterest,impliedVolatility,inTheMoney,expirationDate,dte,CALL,mark
2,VIXY201023C00018500,18.5,0.50,0.75,118.0,97.0,0.742190,0.0,2020-10-24,0.016438,True,0.625
3,VIXY201023C00019000,19.0,0.40,0.65,22.0,311.0,0.832033,0.0,2020-10-24,0.016438,True,0.525
4,VIXY201023C00019500,19.5,0.30,0.50,56.0,12.0,0.859376,0.0,2020-10-24,0.016438,True,0.400
5,VIXY201023C00020000,20.0,0.30,0.40,152.0,120.0,0.943360,0.0,2020-10-24,0.016438,True,0.350
6,VIXY201023C00020500,20.5,0.15,0.25,20.0,46.0,0.863283,0.0,2020-10-24,0.016438,True,0.200
7,VIXY201023C00021000,21.0,0.15,0.30,1.0,200.0,1.013677,0.0,2020-10-24,0.016438,True,0.225
50,VIXY201030C00018500,18.5,0.95,1.50,1.0,23.0,0.946290,0.0,2020-10-31,0.035616,True,1.225
51,VIXY201030C00019000,19.0,0.75,1.25,4.0,27.0,0.929688,0.0,2020-10-31,0.035616,True,1.000
52,VIXY201030C00019500,19.5,0.70,0.95,25.0,669.0,0.929688,0.0,2020-10-31,0.035616,True,0.825
53,VIXY201030C00020000,20.0,0.50,0.85,7.0,190.0,0.927735,0.0,2020-10-31,0.035616,True,0.675


In [30]:
yf.Ticker('VIXY')

yfinance.Ticker object <VIXY>

In [31]:
stock = yf.Ticker('VIXY')

In [34]:
stock.info['previousClose']


18.11

In [1]:
import yfinance as yf
vixy = yf.Ticker('VIXY')
spxl = yf.Ticker('SPXL')

In [48]:
pprint(vixy)

yfinance.Ticker object <VIXY>


In [2]:
import pickle

In [3]:
pickle.dump(vixy, open('vixy.20Oct20.p','wb'))
pickle.dump(spxl, open('spxl.20Oct20.p','wb'))


In [51]:
pprint(vixy)

yfinance.Ticker object <VIXY>


In [4]:
from datetime import date

today = date.today()

# dd/mm/YY
d1 = today.strftime("%d/%m/%Y")
print("d1 =", d1)

# Textual month, day and year	
d2 = today.strftime("%B %d, %Y")
print("d2 =", d2)

# mm/dd/y
d3 = today.strftime("%m/%d/%y")
print("d3 =", d3)

# Month abbreviation, day and year	
d4 = today.strftime("%b-%d-%Y")
print("d4 =", d4)



d1 = 20/10/2020
d2 = October 20, 2020
d3 = 10/20/20
d4 = Oct-20-2020


In [6]:
d = date.today().strftime("%d%b%y")

'20Oct20'

In [8]:
import yfinance as yf
import pickle
vixy = yf.Ticker('VIXY')
spxl = yf.Ticker('SPXL')
d = date.today().strftime("%d%b%Y")
pickle.dump(vixy, open('vixy.'+d+'.p','wb'))
pickle.dump(spxl, open('spxl.'+d+'.p','wb'))


In [9]:
for i in ['z','v']:
    s = 'x'+i+"xx"
    print (s +'\n')
 

xzxx

xvxx



In [13]:
import yfinance as yf
import pickle
d = date.today().strftime("%d%b%y")
for i in ["VIXY","SPXL"]:
    s = yf.Ticker(i)
    pickle.dump(s, open(i+'.'+d+'.p','wb'))

In [1]:



d


NameError: name 'd' is not defined